<a href="https://colab.research.google.com/github/JesusMartinAP/ConsolidadorExcel/blob/main/ControStockChile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Librerias para instalar**

In [1]:
!pip install playwright openpyxl
!playwright install


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 9.3 MB/s eta 0:00:00
161.3 MiB [] 0% 0.0s161.3 MiB [] 0% 33.2s161.3 MiB [] 0% 23.6s161.3 MiB [] 0% 16.5s161.3 MiB [] 0% 11.7s161.3 MiB [] 0% 7.9s161.3 MiB [] 1% 6.3s161.3 MiB [] 2% 5.5s161.3 MiB [] 2% 5.2s161.3 MiB [] 2% 5.3s161.3 MiB [] 3% 5.3s161.3 MiB [] 3% 4.8s161.3 MiB [] 4% 4.6s161.3 MiB [] 4% 4.4s161.3 MiB [] 5% 4.4s161.3 MiB [] 5% 4.3s161.3 MiB [] 6% 4.2s161.3 MiB [] 6% 4.1s161.3 MiB [] 7% 4.1s161.3 MiB [] 8% 3.9s161.3 MiB [] 9% 3.8s161.3 MiB [] 9% 3.7s161.3 MiB [] 10% 3.7s161.3 MiB [] 10% 3.6s161.3 MiB [] 11% 3.5s161.3 MiB [] 12% 3.5s161.3 MiB [] 12% 3.4s161.3 MiB [] 13% 3.5s161.3 MiB [] 14% 3.4s161.3 MiB [] 14% 3.5s161.3 MiB [] 14% 3.4s161.3 MiB [] 15% 3.4s161.3 MiB [] 15% 3.3s161.3 MiB [] 16% 3.3s161.3 MiB [] 17% 3.2s161.3 MiB [] 18% 3.1s161.3 MiB [] 19% 3.0s161.3 MiB [] 20% 2.9s161.3 MiB [] 20% 2.8s161.3 MiB [] 21% 2.7s161.3 MiB [] 22% 2.6s161.3 MiB [] 23% 2.6s161.3 MiB [] 24% 2.5s161.3 MiB [] 25% 2.5s161.3 MiB [] 2

#Script Control de Stock CH

In [10]:
import concurrent.futures
from datetime import datetime
from playwright.sync_api import sync_playwright
from openpyxl import Workbook
from threading import Lock

# Variables globales
proceso_en_ejecucion = False
estado_codigos = []
total_codigos = 0
codigos_procesados = 0
lock = Lock()  # Para controlar el acceso concurrente a las variables globales

# Función para obtener el estado de un producto, su precio y la cantidad de imágenes usando Playwright
def obtener_estado_y_precio(codigo_padre):
    url_base = f'https://www.marathon.cl/{codigo_padre}.html'
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()
        try:
            page.goto(url_base)
            page.wait_for_selector("body", timeout=5000)  # Esperar a que el cuerpo de la página se cargue

            # Verificar si la página redirige al inicio (indicador: URL de redirección)
            if page.url == "https://www.marathon.cl/home/":
                return codigo_padre, "Web no encontrada", "Precio no disponible", "Cantidad de imágenes no disponible"

            # Buscar los botones de talla y determinar si están seleccionados
            botones_talla = page.query_selector_all("button.size-attribute.swatchable.selectable.swatch-square")
            estado = "Agotado"
            for boton in botones_talla:
                if "selected-assistive-text" in boton.inner_html():
                    estado = "Disponible" if not boton.is_disabled() else "Agotado"
                    break

            # Extraer el precio
            precio_element = page.query_selector('span.sales > span.value')
            precio = precio_element.inner_text().strip() if precio_element else "Precio no disponible"

            # Contar la cantidad de imágenes
            imagenes = page.query_selector_all('img.gallery_img')
            cantidad_imagenes = len(imagenes)

            return codigo_padre, estado, precio, cantidad_imagenes
        except Exception as e:
            return codigo_padre, f"Error: {e}", "Precio no disponible", "Cantidad de imágenes no disponible"
        finally:
            browser.close()

# Función para procesar los códigos y guardar en Excel
def procesar_codigos(codigos):
    global proceso_en_ejecucion, estado_codigos, total_codigos, codigos_procesados
    total_codigos = len(codigos)
    start_time = datetime.now()

    def obtener_estado_concurrente(codigo_padre):
        return obtener_estado_y_precio(codigo_padre)

    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:  # Ajustar el número de trabajadores según sea necesario
        futures = {executor.submit(obtener_estado_concurrente, codigo): codigo for codigo in codigos}
        for future in concurrent.futures.as_completed(futures):
            if not proceso_en_ejecucion:
                break
            codigo = futures[future]
            try:
                codigo_padre, estado, precio, cantidad_imagenes = future.result()
            except Exception as exc:
                codigo_padre = codigo  # Inicializar código_padre en caso de error
                estado = f"Error: {exc}"
                precio = "Precio no disponible"
                cantidad_imagenes = "Cantidad de imágenes no disponible"
            with lock:
                estado_codigos.append((codigo_padre, estado, precio, cantidad_imagenes))
                codigos_procesados += 1

            # Actualizar información en la consola
            elapsed_time = datetime.now() - start_time
            tiempo_transcurrido = str(elapsed_time).split('.')[0]  # Formato HH:MM:SS
            print(f"Procesando código {codigos_procesados}/{total_codigos} - Tiempo transcurrido: {tiempo_transcurrido}")

    if proceso_en_ejecucion:
        guardar_resultados()

# Función para pausar el proceso
def pausar_proceso():
    global proceso_en_ejecucion
    proceso_en_ejecucion = False
    print("Proceso pausado. Puede continuar luego.")

# Función para detener el proceso
def detener_proceso():
    global proceso_en_ejecucion
    proceso_en_ejecucion = False
    guardar_resultados()
    print("Proceso detenido.")

# Función para guardar los resultados en Excel
def guardar_resultados():
    global proceso_en_ejecucion
    # Guardar en Excel
    wb = Workbook()
    ws = wb.active
    ws.title = "Control Stock Web"
    ws['A1'] = "CODIGO"
    ws['B1'] = "STATUS WEB"
    ws['C1'] = "PRECIO"
    ws['D1'] = "Cant. Img"
    for i, (codigo, estado, precio, cantidad_imagenes) in enumerate(estado_codigos, start=2):
        ws[f'A{i}'] = codigo
        ws[f'B{i}'] = estado
        ws[f'C{i}'] = precio
        ws[f'D{i}'] = cantidad_imagenes

    # Guardar archivo
    fecha_actual = datetime.now().strftime("%Y-%m-%d")
    nombre_archivo = f"Control Stock Web {fecha_actual}.xlsx"
    wb.save(nombre_archivo)
    print(f"Archivo '{nombre_archivo}' guardado con los estados, precios y cantidad de imágenes de los productos.")
    # Descarga el archivo de Excel en Colab
    from google.colab import files
    files.download(nombre_archivo)

# Función para iniciar el procesamiento
def iniciar_procesamiento(codigos):
    global proceso_en_ejecucion, codigos_procesados, estado_codigos
    if not proceso_en_ejecucion:
        proceso_en_ejecucion = True
        codigos_procesados = 0
        estado_codigos = []
        procesar_codigos(codigos)

# Leer los códigos del archivo de texto "Articulo.txt"
def leer_codigos_desde_archivo(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        codigos = [linea.strip() for linea in archivo if linea.strip()]
    return codigos

# Leer los códigos desde "Articulo.txt"
codigos = leer_codigos_desde_archivo('Articulo.txt')

# Iniciar el procesamiento
iniciar_procesamiento(codigos)


Procesando código 1/5 - Tiempo transcurrido: 0:00:21
Procesando código 2/5 - Tiempo transcurrido: 0:00:24
Procesando código 3/5 - Tiempo transcurrido: 0:00:25
Procesando código 4/5 - Tiempo transcurrido: 0:00:26
Procesando código 5/5 - Tiempo transcurrido: 0:00:26
Archivo 'Control Stock Web 2024-07-09.xlsx' guardado con los estados, precios y cantidad de imágenes de los productos.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Pegar los códigos:

Procesando código 1/10 - Tiempo transcurrido: 0:00:38
Procesando código 2/10 - Tiempo transcurrido: 0:00:38
Procesando código 3/10 - Tiempo transcurrido: 0:00:38
Procesando código 4/10 - Tiempo transcurrido: 0:00:38
Procesando código 5/10 - Tiempo transcurrido: 0:00:38
Procesando código 6/10 - Tiempo transcurrido: 0:00:38
Procesando código 7/10 - Tiempo transcurrido: 0:00:38
Procesando código 8/10 - Tiempo transcurrido: 0:00:38
Procesando código 9/10 - Tiempo transcurrido: 0:00:39
Procesando código 10/10 - Tiempo transcurrido: 0:00:40
Archivo 'Control Stock Web 2024-07-09.xlsx' guardado con los estados, precios y cantidad de imágenes de los productos.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>